In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, PrecisionRecallDisplay, precision_score, recall_score, roc_auc_score, RocCurveDisplay, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [3]:
# URLs of the CSV files
urls = {
    "customers": "https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv",
    "engagements": "https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv",
    "marketing": "https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv",
    "transactions": "https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv"
}

In [4]:
# Read each CSV file into a DataFrame
customers_df = pd.read_csv(urls["customers"])
engagements_df = pd.read_csv(urls["engagements"])
marketing_df = pd.read_csv(urls["marketing"])
transactions_df = pd.read_csv(urls["transactions"])

In [ ]:
# Merge DataFrames on 'customer_id'
merged_df = customers_df.merge(engagements_df, on="customer_id", how="left")
merged_df = merged_df.merge(marketing_df, on="customer_id", how="left")
merged_df = merged_df.merge(transactions_df, on="customer_id", how="left")

In [ ]:
# Display the merged DataFrame
print(merged_df)